In [4]:
import tensorflow as tf
import numpy as np

In [5]:
class RBM:
    def __init__(self, num_visible, num_hidden, learning_rate=0.1, epochs=100):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.learning_rate = learning_rate
        self.epochs = epochs

        self.visible_bias = tf.Variable(tf.random.normal([self.num_visible]), name="visible_bias")
        self.hidden_bias = tf.Variable(tf.random.normal([self.num_hidden]), name="hidden_bias")
        self.weights = tf.Variable(tf.random.normal([self.num_visible, self.num_hidden], stddev=0.1), name="weights")

    def sample_hidden(self, visible_samples):
        hidden_activation = tf.nn.sigmoid(tf.matmul(visible_samples, self.weights) + self.hidden_bias)
        hidden_samples = tf.nn.relu(tf.sign(hidden_activation - tf.random.uniform(tf.shape(hidden_activation))))
        return hidden_samples

    def sample_visible(self, hidden_samples):
        visible_activation = tf.nn.sigmoid(tf.matmul(hidden_samples, tf.transpose(self.weights)) + self.visible_bias)
        visible_samples = tf.nn.relu(tf.sign(visible_activation - tf.random.uniform(tf.shape(visible_activation))))
        return visible_samples

    def train(self, data):
        for epoch in range(self.epochs):
            for batch in data:
                visible_samples = tf.convert_to_tensor(batch, dtype=tf.float32)
                
                with tf.GradientTape() as tape:
                    hidden_samples = self.sample_hidden(visible_samples)
                    reconstructed_visible_samples = self.sample_visible(hidden_samples)
                    positive_gradient = tf.matmul(tf.transpose(visible_samples), hidden_samples)
                    negative_gradient = tf.matmul(tf.transpose(reconstructed_visible_samples), self.sample_hidden(reconstructed_visible_samples))

                    contrastive_divergence = (positive_gradient - negative_gradient) / tf.cast(tf.shape(visible_samples)[0], tf.float32)
                    self.weights.assign_add(self.learning_rate * contrastive_divergence)
                    self.visible_bias.assign_add(self.learning_rate * tf.reduce_mean(visible_samples - reconstructed_visible_samples, axis=0))
                    self.hidden_bias.assign_add(self.learning_rate * tf.reduce_mean(hidden_samples - self.sample_hidden(reconstructed_visible_samples), axis=0))

    def reconstruct(self, data):
        visible_samples = tf.convert_to_tensor(data, dtype=tf.float32)
        hidden_samples = self.sample_hidden(visible_samples)
        reconstructed_visible_samples = self.sample_visible(hidden_samples)
        return reconstructed_visible_samples.numpy()


In [6]:
# Example usage
np.random.seed(123)
data = np.random.randint(0, 2, size=(100, 5))  # Sample binary data
rbm = RBM(num_visible=5, num_hidden=3, learning_rate=0.1, epochs=1000)
rbm.train([data])  # Note: Wrap data in a list to create a batch
reconstructed_samples = rbm.reconstruct(data)
print("Reconstructed Samples:")
print(reconstructed_samples)

Reconstructed Samples:
[[1. 0. 1. 0. 1.]
 [0. 1. 0. 1. 1.]
 [1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 0. 1. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0.]
 [0. 1. 1. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0.]
 [1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 0.]
 [1. 0. 1. 0. 0.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 1. 0.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 1. 0.]
 [0. 0. 1. 1. 0.]
 [1. 0. 1. 0. 1.]
 [0. 1. 0. 1. 1.]
 [0. 1. 0. 1. 0.]
 [1. 1. 0. 1. 0.]
 [1. 0. 0. 1. 1.]
 [1. 1. 0. 1. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 1. 1. 1.]
 [1. 1. 1. 0. 0.]
 [1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1.]
 [0. 1. 1. 1. 1.]
 [0. 0. 0. 1. 0.]
 [1. 1. 1. 0. 1.]
 [0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 1. 1. 1.]
 [0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1.]
 [0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1.]
 [1. 1. 0. 1. 0.]
 [1. 